In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import warnings
warnings.simplefilter("ignore")

# cif2vec

## cell params

In [3]:
from pymatgen.io.cif import CifParser
import fnmatch

In [4]:
main_folder = r"cifs/"

cif_data = pd.DataFrame(columns=["a", "b", "c", "alpha", "beta", "gamma", "volume", "sg_number"])

In [5]:
bad_parse  = []
bad_get_sg = []

In [6]:
for file in os.listdir(main_folder):
    if fnmatch.fnmatch(file, '*.cif'):
        try:
            # print(os.path.join(file))
            struct = CifParser(os.path.join(main_folder, file)).get_structures()[0]
            try:
                d_lattice = struct.as_dict()['lattice']
                property = [d_lattice["a"], d_lattice["b"], d_lattice["b"],
                            d_lattice["alpha"], d_lattice["beta"], d_lattice["gamma"], d_lattice["volume"],
                            struct.get_space_group_info()[1]]
                cif_data.loc[file[:-4]] = property
            except ValueError:
                print(f"Error with get sg: {os.path.join(file)}")
                bad_get_sg.append(os.path.join(file))
        except ValueError:
            print(f"Error with prase: no structure in {file}")
            bad_parse.append(file)
        except KeyError:
            print(f"Error with prase: no parameters in {file}")
            bad_parse.append(file)


Error with prase: no parameters in fromCAU130GPa.cif
Error with prase: no parameters in fromNOTT3000GPa.cif
Error with prase: no parameters in toNOTT3001GPashear.cif


In [7]:
cif_data # all good

,a,b,c,alpha,beta,gamma,volume,sg_number
12fromMnMOF1toacetonsolvated1977803,12.307000,13.006000,13.006000,90.000000,90.000000,92.100000,5060.570412,11.0
1cpdesMOFbacktotheinitial1944823,9.718219,9.718219,9.718219,105.830113,105.830113,113.157513,815.303443,69.0
1from61055570,8.272300,8.634300,8.634300,90.000000,90.000000,92.360000,1451.919669,14.0
1initialMnMOF1DMF989581,12.289000,25.974000,25.974000,90.000000,90.000000,93.570000,10290.612205,14.0
1to71031676,8.359200,8.549800,8.549800,111.219000,106.893000,92.670000,725.669576,2.0
...,...,...,...,...,...,...,...,...
toMOF5C7desolvatedphase2040923,12.786337,12.786337,12.786337,84.507765,84.507765,84.507765,2063.376733,148.0
topart4,17.554724,17.554724,17.554724,90.000000,90.000000,93.289645,12339.495390,20.0
toSTA26ZrCchangedsymmetry1571656,27.968600,28.355728,28.355728,90.748117,90.000000,90.000000,22486.160078,63.0
toVMOP1590348,21.598587,21.598587,21.598587,109.471221,109.471221,109.471221,7756.291592,217.0


## Zeo++ 
notes that zeo++ data was extracted with `-ha` flag

In [8]:
zeo_data = pd.read_csv("preprocessing/zeopp/zeo_data_.csv", index_col=0)

In [9]:
zeo_data

,pld,lcd
toDUT49Ni100K2014970,12.24942,20.69898
from975784,5.51053,10.35538
fromCuIMOF12084812,1.08583,2.60690
initial11455983,1.12479,1.97310
tocompoundwithhexane1403844,1.22634,3.82901
...,...,...
fromPhaseI1031344,1.11924,3.00480
from31483716,2.83690,3.43213
toanie202202073sup0001nku128thf,4.59433,5.90286
finalMOFs1961202,3.20661,4.92448


## mofid

In [10]:
mofid_data = pd.read_csv("preprocessing/mofid/mofid_data_.csv", index_col=0)

## Mordred

In [11]:
from mordred import Calculator, descriptors
from rdkit import Chem

linkers = [smi.strip('"')[1:-1].replace("'", "").split(", ") for smi in mofid_data['linker'].values]
mols = [[Chem.MolFromSmiles(smi) for smi in smi_list] for smi_list in linkers]
calc = Calculator(descriptors, ignore_3D=False)
def f(mof):
    try: return calc.pandas(mof)
    except TypeError:
        return None
    
dfs = [f(mof) for mof in mols]
data_mordred = pd.DataFrame(columns=dfs[0].columns)

for i, filename in enumerate(mofid_data.index):
    try:
        if linkers[i] != [""]:
            data_mordred.loc[filename] = dfs[i].mean()
    except AttributeError:
        print(f"{filename:_^20}")
#data_mordred.to_csv("../data/all_f_main_dataset_mordred_V2.csv")

[14:50:34] Explicit valence for atom # 17 C, 5, is greater than permitted
[14:50:34] Explicit valence for atom # 14 C, 5, is greater than permitted
[14:50:34] Explicit valence for atom # 13 C, 5, is greater than permitted
[14:50:34] Explicit valence for atom # 0 N, 4, is greater than permitted
100%|██████████| 1/1 [00:01<00:00,  1.74s/it]


C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mordred\Autocorrelation.py:97: RuntimeWarning: Mean of empty slice.
  return avec - avec.mean()
C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mordred\Constitutional.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return S / self.mol.GetNumAtoms()


100%|██████████| 1/1 [00:01<00:00,  1.71s/it]


C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mordred\Autocorrelation.py:97: RuntimeWarning: Mean of empty slice.
  return avec - avec.mean()
C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mordred\Constitutional.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return S / self.mol.GetNumAtoms()


100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


...

C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


C:\Users\droid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 1/1 [00:02<00:00,  2.13s/it]


_initialMOF1014506__
tochangedstructure1014507
____from2007086_____


## Node

In [12]:
def metal_from_node(node: str):
    import re
    """
    input: smilesNodes: str
    return: 
    'metals' in node: list
    'unique' types of metals: np.array, dtype='<U2'
    'count' of unique: np.array, dtype=int
    """
    # "O[Zr]123(O)[OH]4[Zr]56([O]3[Zr]37([OH]2[Zr]28([O]1[Zr]14([O]6[Zr]([OH]53)([OH]21)([O]78)(O)O)([OH2])([OH2])O)[OH2])([OH2])([OH2])O)[OH2]"
    node = node.replace("OH", "").replace("O", "")
    node = node.replace("[", "").replace("]", "").replace(")", "").replace("(", "").replace(",", "")
    node = re.sub(r"\d", "", node) # replace numbers
    #print(node)
    # "ZrZrZrZrZrZr"
    start_cut = 0
    metals = []
    for i, char in enumerate(node[1:]):
        if not char.islower():
            metals.append(node[start_cut:i+1])
            start_cut = i+1
    metals.append(node[start_cut:])
    unique, counts = np.unique(np.array(metals), return_counts=True)
    return metals, unique, counts

In [13]:
elemental_descriptors = pd.read_csv("preprocessing/qmof/data/elemental_descriptors.csv")

In [14]:
node_descriptors = pd.DataFrame(columns=("n_metals", 'n_types_metals', 'Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)'))

d = ['Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']

for filename, node in zip(mofid_data.index, mofid_data["node"]):
    try:
       metals, unique, count = metal_from_node(node.replace("'", "").replace(" ", "").replace("%", ""))
       #print(metals)
       n_metals = count.sum()
       n_metals_types = count.__len__()
       df = pd.DataFrame(columns=d, index=[range(count.sum())])
       for metal in metals:
          #print(metal)
          df.loc[len(df)] = elemental_descriptors.loc[elemental_descriptors["Symbol"] == metal].loc[:,d].iloc[0]
       node_descriptors.loc[filename] = n_metals, n_metals_types, *df.mean().array
    except IndexError:
       print(f"Error with {node}")

Error with ['*']
Error with []


Error with ['*']
Error with []
Error with ['*']
Error with []
Error with ['*']
Error with []
Error with []


In [15]:
node_descriptors

,n_metals,n_types_metals,Atomic_Number,Atomic_Weight,Atomic Radius,Mulliken EN,polarizability(A^3),electron affinity(kJ/mol)
fromCuICuIIMOF22084857,1.0,1.0,29.000000,63.546000,1.450000,4.480000,6.700000,119.0
toja054913asi004,1.0,1.0,47.000000,107.868200,1.650000,4.440000,7.900000,126.0
fromMOF2fa1052168,2.0,1.0,27.000000,58.933200,1.520000,4.300000,7.500000,64.0
initialMOF1014506,2.0,2.0,37.000000,87.477384,2.105000,2.975000,25.650000,68.0
initialsMOFs1961201,2.0,1.0,29.000000,63.546000,1.450000,4.480000,6.700000,119.0
...,...,...,...,...,...,...,...,...
tochangedstructure1014507,3.0,2.0,45.666667,108.973256,2.173333,3.016667,26.333333,73.0
1from61055570,1.0,1.0,29.000000,63.546000,1.450000,4.480000,6.700000,119.0
from2007086,6.0,2.0,22.500000,47.073050,1.140000,6.350000,4.500000,230.5
toVMOP1590348,1.0,1.0,23.000000,50.941500,1.710000,3.600000,12.400000,51.0


In [16]:
import json

mapping_ = json.load(open("names_mapping.json"))
t = pd.read_csv("database/main.csv", sep=";")
# t.index = [mapping_[i+".cif"][:-4] for i in t.index]

In [17]:
t['Stimuli'].unique()

array(['solvent', 'gas', 'T', 'humidity', 'Pressure', 'pressure',
       'solvent, gas'], dtype=object)

In [18]:
target = pd.DataFrame(columns=['target'])
for i in t.index:
    ans = 0 if t.loc[i]["Stimuli"].find("T") != -1 or t.loc[i]["Stimuli"].find("pressure") != -1 or t.loc[i]["Stimuli"].find("Pressure") != -1 else 1     
    target.loc[mapping_[t.loc[i]["CIF name"]][:-4]] = [ans]

# concatenate

In [19]:
index = list({*list(zeo_data.index)} & {*list(cif_data.index)} & {*list(data_mordred.index)} & {*list(node_descriptors.index)} & {*list(target.index)})

In [20]:
marked_dataset = pd.concat([zeo_data.loc[index], cif_data.loc[index], data_mordred.loc[index], node_descriptors.loc[index], target.loc[index]], axis=1)

In [21]:
import joblib

In [22]:
from preproc_model import PreprocessingModel
preproc = joblib.load("preprocessing/preproc_m.pkl")

In [23]:
x, y = marked_dataset.drop(['target'], axis=1), marked_dataset["target"]

In [24]:
np.unique(y)

array([0, 1], dtype=int64)

In [25]:
np.unique(y, return_counts=True), sum(np.unique(y, return_counts=True)[1])

((array([0, 1], dtype=int64), array([37, 46], dtype=int64)), 83)

In [26]:
x = preproc.transform(x.rename({ "sg_number": "spacegroupNumber"}, axis=1))

In [27]:
x.to_csv("preprocessing/datasets/main_dataset.csv")
y.to_csv("preprocessing/datasets/main_target.csv")

In [28]:
added = pd.read_csv("database/t_solvent.csv", sep=";")